# Libraries

In [4]:
# Common
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt 
import seaborn as sns
from sqlalchemy import create_engine


In [5]:
load_dotenv(dotenv_path=".env")

True

# Exploratory

## Connect to db

In [11]:
connection_string = f"mysql+pymysql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
db_connection = create_engine(connection_string)

## Table at glance

In [12]:
with open('./queries/tb_overview.sql', 'r') as query:
    tb_overview = pd.read_sql_query(query.read(), con=db_connection)

In [26]:
tb_overview

,column_name,data_type
0,bid,int
1,paidon,datetime
2,paygate_code,varchar
3,status_id,varchar
4,seller_id,int
5,operator_id,int
6,class_id,int
7,class_name,varchar
8,from_id,int
9,from_province_id,int


In [20]:
print("The table has: ", tb_overview.shape[0],"columns")

The table has:  43 columns


In [18]:
tb_overview.groupby(['data_type']).agg('count').sort_values(by='column_name', ascending=False)

,column_name
data_type,
varchar,12
int,11
float,8
char,5
datetime,5
date,1
tinyint,1


In [28]:
print("The table has ", tb_overview[tb_overview['column_name'].str.contains('_id')].shape[0]," fields with '_id' in the name")
print("The table has ", tb_overview[tb_overview['column_name'].str.contains('_usd')].shape[0]," fields with '_usd' in the name")

The table has  14  fields with '_id' in the name
The table has  8  fields with '_usd' in the name


So we have:
- 17 string fields to extract features, categorize data
- 8 fields (float - usd) indicate value in USD
- 6 fields related to date/datetime, answering seasonality questions
- 15 id fields (14 _id and bid) to consider droping/masking

Categorizing fields into category:
- Booking: status_id, seats
- trend/seasonality/timing: createdon,paidon,godate --refund_date
- financial related: *_usd --> re-check with the formular in the file | paygate_code 
- route/services: from_, to_station_name | from_,to_country_id --> international or domestics travel, province look like it is dropable || vehical_id,class_name || trip_duration
- operator: seller_id, operator_id --> shoud I map with operator table to get more info?
- channel & customer info: cust_id, channel, referrer, landing --> segmentation? | user_agent --> better review to extract more info.